In [1]:
import os
import numpy as np

from mtuq import read, open_db, download_greens_tensors
from mtuq.event import Origin
from mtuq.graphics import plot_data_greens2, plot_beachball, plot_misfit_dc, plot_misfit_lune
from mtuq.grid import DoubleCoupleGridRegular, FullMomentTensorGridSemiregular
from mtuq.grid_search import grid_search
from mtuq.misfit import Misfit
from mtuq.process_data import ProcessData
from mtuq.util import fullpath, merge_dicts, save_json
from mtuq.util.cap import parse_station_codes, Trapezoid

- In this project, we utilize MTUQ (https://github.com/uafgeotools/mtuq?tab=readme-ov-file) to estimate the moment tensor and PySEP (https://pysep.readthedocs.io/en/latest/) for seismic data curation. 

- Basic ideas: 
    - MTUQ employs a moment-tensor parameterization, exploring the moment-tensor space in a uniform manner (Tape & Tape, 2015). Its underlying algorithm is from Zhu & Hemberger (1996). Namely, they introduced the cut-and-paste (CAP) method by splitting the seismic data into P-waves and surface waves. These two are inverted independently. The time-shift correlation is performed to find a maximum and calculate the misfit for synthetic and observed data. This approch takes into account the discrepency between the 1D and 3D velocity structure model. 
    - PySEP is used to access seismic data (waveforms, stations, metadata, etc.). We also construct yaml file as input for inversion. 

#### M 4.6 - 2 km S of Roosevelt, Washington

In [1]:
%cd ~/Desktop/MTUQ_project/content/workdir/data
!cp pysep_config.yaml 2019_07_12_WASHINGTON.yaml

/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/data


We need to fill in an empty PySEP config file:

- get data from the UW network

- within 250 km of epicentral distance

- request the HH? channels (high-gain broadband stations)

In [2]:
%cd ~/Desktop/MTUQ_project/content/workdir/data
!pysep -c ./2019_07_12_WASHINGTON.yaml -o

/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/data
[2024-07-13 15:41:49] - pysep - DEBUG: running PySEP version 0.6.1
[2024-07-13 15:41:49] - pysep - INFO: overwriting default parameters with config file: './2019_07_12_WASHINGTON.yaml'
[2024-07-13 15:41:49] - pysep - DEBUG: event_tag: None -> 2019_07_12_WASHINGTON
[2024-07-13 15:41:49] - pysep - DEBUG: config_file: ./2019_07_12_WASHINGTON.yaml -> None
[2024-07-13 15:41:49] - pysep - DEBUG: origin_time: None -> 2019-07-12T09:51:38.340Z
[2024-07-13 15:41:49] - pysep - DEBUG: seconds_before_event: 20 -> 100
[2024-07-13 15:41:49] - pysep - DEBUG: seconds_after_event: 20 -> 300
[2024-07-13 15:41:49] - pysep - DEBUG: event_latitude: None -> 47.873
[2024-07-13 15:41:49] - pysep - DEBUG: event_longitude: None -> -122.016
[2024-07-13 15:41:49] - pysep - DEBUG: event_depth_km: None -> 28.8
[2024-07-13 15:41:49] - pysep - DEBUG: event_magnitude: None -> 4.58
[2024-07-13 15:41:49] - pysep - DEBUG: networks: * -> UW
[2024-07-13 15:41:

Before opening the data in `MTUQ`, one should **always have a look at the record sections**.

In [3]:
# Raw vertical component - recsec_1.png
!recsec --pysep_path ./2019-07-12T095138_WASHINGTON/SAC -o --scale_by 'geometric_spreading'  --integrate 1 --component Z --save ./2019-07-12T095138_WASHINGTON/recsec_1.png

# Raw radial component - recsec_2.png
!recsec --pysep_path ./2019-07-12T095138_WASHINGTON/SAC -o --scale_by 'geometric_spreading'  --integrate 1 --component R --save ./2019-07-12T095138_WASHINGTON/recsec_2.png

# Raw transverse component - recsec_3.png
!recsec --pysep_path ./2019-07-12T095138_WASHINGTON/SAC -o --scale_by 'geometric_spreading'  --integrate 1 --component T --save ./2019-07-12T095138_WASHINGTON/recsec_3.png

[2024-07-13 15:47:13] - pysep - INFO: starting record section plotter
[2024-07-13 15:47:13] - pysep - INFO: attempting to read 130 'data' files from: ./2019-07-12T095138_WASHINGTON/SAC
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHINGTON.UW.GBB..HHE.sac
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHINGTON.UW.GBB..HHR.sac
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHINGTON.UW.WISH..HHR.sac
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHINGTON.UW.WISH..HHE.sac
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHINGTON.UW.SP2..HHN.sac
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHINGTON.UW.FORK..HHZ.sac
[2024-07-13 15:47:14] - pysep - DEBUG: ./2019-07-12T095138_WASHINGTON/SAC/2019-07-12T095138_WASHIN

![recsec_1](https://github.com/Benz-Poobua/Assessing-MTUQ-with-PNSN-catalog/raw/main/WA_eq_demo/recsec_1.png)


![recsec_2](https://github.com/Benz-Poobua/Assessing-MTUQ-with-PNSN-catalog/raw/main/WA_eq_demo/recsec_2.png)

![recsec_3](https://github.com/Benz-Poobua/Assessing-MTUQ-with-PNSN-catalog/raw/main/WA_eq_demo/recsec_3.png)


If one open the record sections, one will observe a few stations with significant, long-period oscillations. we can discard them.

Open the `weights.dat` file, and one sets these stations' weights to 0 (for all 5 data groups). By discarding them now, one won't have to download their Green's functions.

In [14]:
# ! nano ~/Desktop/MTUQ_project/content/workdir/data/2019-07-12T095138_WASHINGTON/weights.dat

In [8]:
%cd ~/Desktop/MTUQ_project/content/workdir/2019_07_12/

path_data=    '/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/data/2019-07-12T095138_WASHINGTON/SAC/*'
path_weight= '/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/data/2019-07-12T095138_WASHINGTON/weights.dat'
model = 'ak135' # model name for the taup-computed pick
event_id = '2019-07-12T095138_WASHINGTON'

# define the data-processing

process_bw = ProcessData(
    filter_type='Bandpass',
    freq_min= 0.1,
    freq_max= 0.333,
    pick_type='taup',
    taup_model=model,
    window_type='body_wave',
    window_length=15,
    capuaf_file=path_weight,
    )

process_sw = ProcessData(
    filter_type='Bandpass',
    freq_min=0.025,
    freq_max=0.0625,
    pick_type='taup',
    taup_model=model,
    window_type='surface_wave',
    window_length=150,
    capuaf_file=path_weight,
    )

/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/2019_07_12


In [10]:
# define the misfit
#
# for our objective function, we will use a sum of body and surface wave
# contributions
#

misfit_bw = Misfit(
    norm='L2',
    time_shift_min=-2,
    time_shift_max=+2,
    time_shift_groups=['ZR'],
    )

misfit_sw = Misfit(
    norm='L2',
    time_shift_min=-10,
    time_shift_max=+10,
    time_shift_groups=['ZR','T'],
    )

# User-supplied weights control how much each station contributes to the objective function

station_id_list = parse_station_codes(path_weight)

#
# Next, we specify the moment tensor grid and source-time function
#

grid = DoubleCoupleGridRegular(
    npts_per_axis=40, 
    magnitudes=[4.5])

# or
# grid = FullMomentTensorGridSemiregular(
#     npts_per_axis=40, 
#     magnitudes = [4.5])

wavelet = Trapezoid(
    magnitude= 4.5)

origin = Origin({
    'time': '2019-07-12T09:51:38.340Z',
    'latitude': 47.873,
    'longitude': -122.016,
    'depth_in_m': 28800,
    })

# data import
print('Reading data...\n')
data = read(path_data, format='sac',
    event_id=event_id,
    station_id_list=station_id_list,
    tags=['units:m', 'type:velocity'])


data.sort_by_distance()
stations = data.get_stations()


print('Processing data...\n')
data_bw = data.map(process_bw)
data_sw = data.map(process_sw)

# Green's function import
print('Reading Greens functions...\n')
greens = download_greens_tensors(stations, origin, model)

print('Processing Greens functions...\n')
greens.convolve(wavelet)
greens_bw = greens.map(process_bw)
greens_sw = greens.map(process_sw)


print('Evaluating body wave misfit...\n')
results_bw = grid_search(
    data_bw, greens_bw, misfit_bw, origin, grid)
print('Evaluating surface wave misfit...\n')
results_sw = grid_search(
    data_sw, greens_sw, misfit_sw, origin, grid)

# combining the two misfit contributions
results = results_bw + results_sw

# Collect information about best-fitting source

# index of the best source solution
source_idx = results.source_idxmin()
best_mt = grid.get(source_idx)

# dictionary of lune parameters
lune_dict = grid.get_dict(source_idx)

# generate figures and save results

print('Generating figures...\n')

# waveform plot
plot_data_greens2(event_id+'_waveforms.png',
    data_bw, data_sw, greens_bw, greens_sw, process_bw, process_sw,
    misfit_bw, misfit_sw, stations, origin, best_mt, lune_dict)

# beachball plot
plot_beachball(event_id+'_beachball.png',
    best_mt, stations, origin)

# Double-couple misfit plot
plot_misfit_dc(event_id+'DC_misfit.png', results)

# Lune
# plot_misfit_lune(event_id+'FMT_misfit.png', results)
# plot_misfit_lune(event_id+'FMT_misfit_tradeoffs.png', results, show_tradeoffs=True)

Reading data...

Processing data...

Reading Greens functions...

Processing Greens functions...

Evaluating body wave misfit...

  Number of misfit evaluations: 6.400e+04

  about 0 percent finished
  about 25 percent finished
  about 50 percent finished
  about 75 percent finished
  Elapsed time (s): 0.467332

Evaluating surface wave misfit...

  Number of misfit evaluations: 6.400e+04

  about 0 percent finished
  about 25 percent finished
  about 50 percent finished
  about 75 percent finished
  Elapsed time (s): 2.579997

Generating figures...



/Users/suphakornpoobua/mtuq/mtuq/util/__init__.py:264: Warning: Nothing to plot: all values are identical
  warnings.warn(*args, **kwargs)
/Users/suphakornpoobua/mtuq/mtuq/util/__init__.py:264: Warning: Nothing to plot: all values are identical
  warnings.warn(*args, **kwargs)


In [11]:
# index of best-fitting moment tensor
idx = results.source_idxmin()

# MomentTensor object
best_mt = grid.get(idx)

# dictionary of lune parameters
lune_dict = grid.get_dict(idx)

# dictionary of Mij parameters
mt_dict = best_mt.as_dict()

# dictionary containing recap info
merged_dict = merge_dicts(
    mt_dict, lune_dict, {'M0': best_mt.moment()},
    {'Mw': best_mt.magnitude()}, origin)

In [12]:
print('Best-fitting source parameters:')
for key, val in merged_dict.items():
  print(f'{key}: {val}')

Best-fitting source parameters:
Mrr: 5800435174432632.0
Mtt: -4103704486655639.5
Mpp: -1696730687776992.0
Mrt: 1609452545268798.5
Mrp: 3326890107715690.0
Mtp: -3126982763259155.0
rho: 1.001186529700906e+16
v: 0.0
w: 0.0
kappa: 40.5
sigma: 74.25
h: 0.48750000000000004
M0: 7079457843841374.0
Mw: 4.5
time: 2019-07-12T09:51:38.340000Z
latitude: 47.873
longitude: -122.016
depth_in_m: 28800.0


In [15]:
# import matplotlib.pyplot as plt
# from IPython.display import Image, display

# image_paths = [
#     '/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/2019_07_12/2019-07-12T095138_WASHINGTON_beachball.png',
#     '/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/2019_07_12/2019-07-12T095138_WASHINGTONDC_misfit.png',
#     '/Users/suphakornpoobua/Desktop/MTUQ_project/content/workdir/2019_07_12/2019-07-12T095138_WASHINGTON_waveforms.png'
# ]

# fig, axes = plt.subplots(1, 3, figsize=(18, 6))
# for ax, img_path in zip(axes, image_paths):
#     ax.imshow(plt.imread(img_path))
#     ax.axis('off')
# plt.show()

Here are the results from the MTUQ:

![Misfit](https://github.com/Benz-Poobua/Assessing-MTUQ-with-PNSN-catalog/raw/393e31e8670439ab3edc613144540435c1c34184/WA_eq_demo/2019-07-12T095138_WASHINGTONDC_misfit.png)

![Beachball](https://raw.githubusercontent.com/Benz-Poobua/Assessing-MTUQ-with-PNSN-catalog/main/WA_eq_demo/2019-07-12T095138_WASHINGTON_beachball.png)

![Waveforms](https://github.com/Benz-Poobua/Assessing-MTUQ-with-PNSN-catalog/raw/main/WA_eq_demo/2019-07-12T095138_WASHINGTON_waveforms.png)
